In [66]:

import regex as re
import pandas as pd
import numpy as np
from termcolor import colored
from urllib.request import urlopen
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [67]:
%pwd 
%cd c:\\Users\\Lenovo\\Flask apps\\House-price-prediction\\

c:\Users\Lenovo\Flask apps\House-price-prediction


In [68]:
MAIN_DF_PATH = r'Dataset\calgary_cleaned_rentals_dataset.csv'
df = pd.read_csv(MAIN_DF_PATH)

In [69]:

df.dropna(subset=['price', 'sq_feet'], inplace=True)
df.reset_index(inplace=True, drop=True)

assert len(df.rented.unique()) ==1
df.drop(columns=['rented'], inplace=True)


In [70]:
# handling second rentals --ADD
def handle_second_rentals(df, info_cols = ['price', 'sq_feet', 'beds', 'baths']):
    """
    separates residence options that has two listings
    creates the second dataset
    then renames both dataset's cols to have info_cols names
    """
    info_cols_2 = [col + '2' for col in info_cols]

    # check for where at least two second house columns are not null
    df_second = df[(~df[info_cols_2[0]].isna())&(~df[info_cols_2[1]].isna())]

    df_second = df_second.drop(columns=info_cols)
    for col in info_cols:
        df_second = df_second.rename(columns={col + '2': col})

    df = df.drop(columns=info_cols_2)
    return df, df_second

In [71]:
df, df_second = handle_second_rentals(df, info_cols = ['price', 'sq_feet', 'beds', 'baths'])

In [72]:
df_total = pd.concat([df, df_second], axis=0)
df_total.reset_index(inplace=True, drop=True)

In [73]:
len(df_total)

2213

In [13]:
df_total.columns

Index(['availability', 'a', 'city', 'community', 'latitude', 'longitude',
       'link', 'type', 'cats', 'dogs', 'sq_feet2', 'price2', 'beds2', 'baths2',
       'utility_heat', 'utility_electricity', 'utility_water', 'utility_cable',
       'utility_internet', 'sq_feet', 'price', 'beds', 'baths'],
      dtype='object')

In [74]:
# --ADD
def clean_undefined_dates(df, null_substitute = 'call for availability'):
    """
    call for availability is represented as 3000-01-01 in the dataset
    so we replace these dates with a string
    """
    for i in range(len(df)):
        try: 
            df.loc[i, 'a'] = pd.to_datetime(df.loc[i, 'a'], format='%Y-%m-%d')
        except: 
            df.loc[i, 'a'] = null_substitute
            df.loc[i, 'availability'] = null_substitute
    df.reset_index(inplace=True)
    return df


df_total = clean_undefined_dates(df_total)

In [75]:
df_total.isna().sum()

index                  0
availability           0
a                      0
city                   1
community              0
latitude               0
longitude              0
link                   0
type                   0
price                  0
beds                   0
sq_feet                0
baths                  0
cats                   0
dogs                   0
utility_heat           0
utility_electricity    0
utility_water          0
utility_cable          0
utility_internet       0
dtype: int64

In [78]:
# -- ADD
def create_month_col(df):
    for i in range(len(df)):
        try:
            df.loc[i, 'available_month'] = int(df.loc[i, 'a'].month)
        except:
            df.loc[i, 'available_month'] = df.loc[i, 'a']
            
    return df

In [79]:
df_total.available_month.unique()

array([1, 5, 'call for availability', 4, 6, 7, 8, 9, 11, 10], dtype=object)

In [ ]:
df_exp =df.dropna('')

In [105]:
len(unwanted)

184

In [107]:
len('+ / -  1000 sq feet')

19

In [181]:
re.findall('[\w\.\!\- ]*\d+ *\+ *\d+ [\w\.\!\- ]*', '996 + 900 SQ FT BALCONY!')#[0]

['996 + 900 SQ FT BALCONY!']

In [152]:
re.findall('\d+', '650  balcony')

['650']

In [187]:
re.findall('[\w\.\!\- ]*\d+ *- *\d+[\w\.\!\- ]*', df_total.loc[1, 'sq_feet'])#[0]

[]

In [218]:
# --ADD

def clean_sq_feet_col(df):

    unwanted = []
    for i in range(len(df)):
        record = df.loc[i, 'sq_feet']

        try:
            df.loc[i, 'sq_feet'] = int(float(record))
            

        except:
            # lengthy descriptions
            if len(record)>30 or not re.findall('\d+', record):
                unwanted.append(i)

            # it is 700-800 balcony
            elif re.findall('[\w\.\!\- ]*\d+ *- *\d+[\w\.\!\- ]*', record):
                num1 = re.findall('\d+', record)[0]
                num2 = re.findall('\d+', record)[1]

                df.loc[i, 'sq_feet'] = (int(num1) + int(num2))/2

            # it is 700 + 500 balcony
            elif re.findall('[\w\.\!\- ]*\d+ *\+ *\d+[\w\.\!\- ]*', record):
                num1 = re.findall('\d+', record)[0]
                num2 = re.findall('\d+', record)[1]

                print('num1 = ', num1)
                print('num2 = ', num2)
                print('record is ', record)
                df.loc[i, 'sq_feet'] = int(num1) + int(num2)

            # ~600
            elif len(re.findall('\d+', record))==1:
                df.loc[i, 'sq_feet'] = int(re.findall('\d+', record)[0])

            # 2,000
            elif re.findall('\d+,*\d+', record):
                df.loc[i, 'sq_feet'] = int(re.findall('\d+,*\d+', '2,000')[0].replace(',', ''))

            else:
                print(df.loc[i, 'sq_feet']) 
                unwanted.append(i)
    df.drop(unwanted, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df
    
df_total = clean_sq_feet_col(df_total)


In [219]:
df_total

,index,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet,available_month
0,0,Immediate,2000-01-01 00:00:00,Calgary,Cliff Bungalow,51.036696,-114.074492,/ab/calgary/rentals/apartment/studio/cliff-bungalow/357976,Apartment,525.0,studio,420,1,0,0,1,1,1,0,0,1
1,1,May 01,2022-05-01 00:00:00,Calgary,Thorncliffe,51.107160,-114.071470,/ab/calgary/rentals/apartment/3-bedrooms/thorncliffe/131827,Apartment,500.0,3,1100,1,0,0,1,1,1,0,0,5
2,2,Immediate,2000-01-01 00:00:00,Calgary,Forest Heights,51.044627,-113.970320,/ab/calgary/rentals/apartment/1-bedroom/forest-heights/pet-friendly/430315,Apartment,999.0,1,522,1,2,2,1,0,1,0,0,1
3,3,Immediate,2000-01-01 00:00:00,Calgary,Lower Mount Royal,51.036826,-114.088533,/ab/calgary/rentals/apartment/1-bedroom/lower-mount-royal/pet-friendly/374579,Apartment,800.0,1,630,1,2,0,1,0,1,0,0,1
4,4,Immediate,2000-01-01 00:00:00,Calgary,Beltline,51.041265,-114.066508,/ab/calgary/rentals/apartment/studio/beltline/pet-friendly/339295,Apartment,695.0,studio,205,1,2,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,2208,Immediate,2000-01-01 00:00:00,Calgary,Livingston,51.184966,-114.074045,/ab/calgary/rentals/townhouse/1-bedroom/livingston/pet-friendly/482132,Townhouse,2300.0,3,505,2.5,2,2,0,0,0,0,0,1
2195,2209,Immediate,2000-01-01 00:00:00,Calgary,Victoria Park,51.039853,-114.066519,/ab/calgary/rentals/apartment/1-bedroom/victoria-park/pet-friendly/368446,Townhouse,2800.0,0,1491,2.0,2,2,1,0,1,0,0,1
2196,2210,Apr 15,2022-04-15 00:00:00,Calgary,Thorncliffe,51.108820,-114.061987,/ab/calgary/rentals/townhouse/2-bedrooms/thorncliffe/pet-friendly/27543,Townhouse,1839.0,3,982,0.0,2,2,1,0,1,0,0,4
2197,2211,May 01,2022-05-01 00:00:00,Calgary,Capitol Hill,51.073473,-114.093629,/ab/calgary/rentals/townhouse/3-bedrooms/capitol-hill/pet-friendly/469604,Townhouse,3650.0,0,1502,3.5,2,2,0,0,0,0,0,5


In [95]:
import plotly.express as px
fig = px.scatter(df_total, x="sq_feet", y="price", color="type")
fig.show()

In [58]:
df_total.loc[0, 'a'].month

1

In [65]:
for i in range(len(df_total)):
    try: 
        try:
            df_total.loc[i, 'available_month'] = df_total.loc[i, 'a'].month
        except: 
            df_total.loc[i, 'available_month'] = df.loc[i, 'a']
    except: print(df_total.loc[i, 'available_month'])

nan
nan
nan
nan
nan
nan
nan
nan


In [64]:
df_total.isna().sum() 

index                  0
availability           0
a                      0
city                   1
community              0
latitude               0
longitude              0
link                   0
type                   0
price                  0
beds                   0
sq_feet                0
baths                  0
cats                   0
dogs                   0
utility_heat           0
utility_electricity    0
utility_water          0
utility_cable          0
utility_internet       0
available_month        8
dtype: int64

In [62]:
df_total.available_month

0       1
1       5
2       1
3       1
4       1
       ..
2208    1
2209    1
2210    4
2211    5
2212    5
Name: available_month, Length: 2213, dtype: object

### unwanted

In [51]:
df_total.a

0       2000-01-01 00:00:00
1       2022-05-01 00:00:00
2       2000-01-01 00:00:00
3       2000-01-01 00:00:00
4       2000-01-01 00:00:00
               ...         
2208    2000-01-01 00:00:00
2209    2000-01-01 00:00:00
2210    2022-04-15 00:00:00
2211    2022-05-01 00:00:00
2212    2022-05-15 00:00:00
Name: a, Length: 2213, dtype: object

In [52]:
df.a

0       2000-01-01
1       2022-05-01
2       2000-01-01
3       2000-01-01
4       2000-01-01
           ...    
2028    3000-00-00
2029    3000-00-00
2030    3099-01-01
2031    3099-01-01
2032    3099-01-01
Name: a, Length: 2033, dtype: object

In [24]:
df_total.columns
checking_cols = [
    'availability', 'city', 'community', 'type', 'beds',
     'baths', 'cats', 'dogs'
     ]

In [25]:
for col in checking_cols:
    print('\ncolumn ', colored(col, 'green'))
    print(df_total.loc[:, col].unique())


column  availability
['Immediate' 'May 01' nan 'Apr 15' 'Apr 30' 'May 15' 'Apr 17' 'Apr 20'
 'Apr 21' 'Apr 28' 'May 06' 'May 07' 'May 09' 'May 10' 'May 13' 'May 20'
 'Jun 01' 'Jun 15' 'Jul 01' 'Jul 15' 'Aug 01' 'Sep 01' 'Negotiable'
 'Apr 07' 'Apr 13' 'Apr 14' 'Jun 30' 'Nov 01' 'Apr 11' 'Apr 09' 'Apr 12'
 'Apr 16' 'Apr 10' 'May 05' 'May 31' 'Jul 08' 'Oct 01' 'Apr 18' 'May 02'
 'May 16' 'Apr 08' 'Jun 05' 'Sep 24' 'Jun 20' 'Jun 22' 'Jun 02' 'Apr 25'
 'May 03' 'May 18' 'May 21' 'May 28' 'Jun 07' 'Jun 16']

column  city
['Calgary' nan]

column  community
['Cliff Bungalow' 'Thorncliffe' 'Forest Heights' 'Lower Mount Royal'
 'Beltline' 'Crescent Heights' 'Braeside' 'Mission' 'Bridgeland' 'Sunalta'
 'Dover' 'Haysboro' 'Downtown' 'Triwood' 'Connaught' 'Bankview' 'Altadore'
 'Scarboro' 'Brentwood' 'Huntington Hills' 'Renfrew' 'South Calgary'
 'Greenview' 'Woodlands' 'Shaganappi' 'Windsor Park' 'Forest Lawn'
 'Killarney' 'Bowness' 'Mount Royal' 'Inglewood' 'Deer Run' 'Dalhousie'
 'Willow Park' 

In [45]:
# --ADD
def clean_baths_col(df):
    for i in range(len(df)):
        try: df.loc[i, 'baths'] = np.floor(float(df.loc[i, 'baths']))
        except: 
            df.loc[i, 'baths'] = 0
    return df

df_total = clean_baths_col(df_total)

In [40]:
np.floor(3.5)

3.0

In [39]:
df_total[df_total.baths.isna()]

,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet


In [29]:
int(re.findall('\d', '3+den')[0])

3

In [32]:
df_total.reset_index(inplace=True, drop=True)

In [36]:
# --ADD
df_total.loc[:, 'cats'] = df_total.loc[:, 'cats'].apply(lambda x : int(x))
df_total.loc[:, 'dogs'] = df_total.loc[:, 'dogs'].apply(lambda x : int(x))

In [35]:
# -- ADD

def clean_beds_col(df):

    for i in range(len(df)):
        val = df.loc[i, 'beds']
        try: df.loc[i, 'beds'] = int(val)
        except: 
            if re.findall('\d', val): 
                df.loc[i, 'beds'] = int(re.findall('\d', val)[0])
            else:
                df.loc[i, 'beds'] = 0
    return df

df_total = clean_beds_col(df_total)


In [27]:
df_total[df_total.beds=='studio']


,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet
0,Immediate,2000-01-01,Calgary,Cliff Bungalow,51.036696,-114.074492,/ab/calgary/rentals/apartment/studio/cliff-bungalow/357976,Apartment,525.0,studio,420,1,0,0,1,1,1,0,0
4,Immediate,2000-01-01,Calgary,Beltline,51.041265,-114.066508,/ab/calgary/rentals/apartment/studio/beltline/pet-friendly/339295,Apartment,695.0,studio,205,1,2,2,0,0,0,0,0
5,Immediate,2000-01-01,Calgary,Crescent Heights,51.064555,-114.064615,/ab/calgary/rentals/apartment/studio/crescent-heights/3898,Apartment,895.0,studio,Approx 345,1,0,0,1,0,1,0,0
9,Immediate,2000-01-01,Calgary,Mission,51.036992,-114.075232,/ab/calgary/rentals/apartment/1-bedroom/mission/pet-friendly/466381,Apartment,875.0,studio,350,1,2,2,1,0,1,0,0
14,Immediate,2000-01-01,Calgary,Haysboro,50.977276,-114.076662,/ab/calgary/rentals/apartment/1-bedroom/haysboro/pet-friendly/75289,Apartment,950.0,studio,374,1,2,2,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,NaN,3099-01-01,Calgary,Scarboro,51.040496,-114.111504,/ab/calgary/rentals/basement/studio/scarboro/481472,Basement,900.0,studio,350,1,0,0,1,1,1,0,1
952,May 01,2022-05-01,Calgary,Banff Trail,51.067351,-114.116672,/ab/calgary/rentals/condo/studio/banff-trail/pet-friendly/484052,Condo,1000.0,studio,255,1,2,2,1,1,1,0,1
1039,Apr 16,2022-04-16,Calgary,Connaught,51.039142,-114.080065,/ab/calgary/rentals/condo/studio/connaught/80161,Condo,1175.0,studio,400,1,0,0,1,0,1,0,0
1235,Apr 15,2022-04-15,Calgary,Beltline,51.038769,-114.079079,/ab/calgary/rentals/condo/studio/beltline/pet-friendly/295734,Condo,1600.0,studio,450,1,2,2,1,1,1,0,1


In [ ]:
len(df[(~df.price2.isna())&(~df.sq_feet2.isna())&((~df.beds.isna()))&(~df.baths2.isna())])

180

In [43]:
df[['price', 'price2']]

,price,price2
0,999.0,NaN
1,974.0,999.0
2,899.0,950.0
3,849.0,999.0
4,800.0,1000.0
...,...,...
182,1245.0,2300.0
183,2700.0,2800.0
184,1567.0,1839.0
185,2925.0,3650.0


In [27]:
df[df.availability != 'Immediate'].availability.unique()
#df.loc[:, 'availability_month'] 

array(['May 01', 'Apr 15', 'Apr 30', 'call for availability', 'May 15',
       'Apr 07', 'Apr 17', 'Apr 20', 'Apr 21', 'Apr 28', 'May 06',
       'May 07', 'May 09', 'May 10', 'May 13', 'May 20', 'Jun 01',
       'Jun 15', 'Jul 01', 'Jul 15', 'Aug 01', 'Sep 01', 'Apr 13',
       'Apr 14', 'Jun 30', 'Nov 01', 'Apr 11', 'Apr 09', 'Apr 10',
       'Apr 12', 'Apr 16', 'May 26', 'Apr 08', 'May 05', 'May 31',
       'Jul 08', 'Oct 01', 'Apr 18', 'May 02', 'May 16', 'Jun 05',
       'Sep 24', 'Jun 20', 'Jun 22', 'Jun 02', 'Apr 25', 'May 03',
       'May 18', 'May 21', 'May 28', 'Jun 07', 'Jun 16'], dtype=object)